<a href="https://colab.research.google.com/github/IgorSouza21/SecundaryStructurePrediction/blob/master/BiologiaComputacional2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

#Imports


In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import SGD
import gc
!pip install -q SwarmPackagepy

#Dataset

In [0]:
# import numpy as np
import glob
import pickle
import pandas as pd

def create_base(base, window, path):
    res = base["RES"]
    dssp = base["DSSP"]

    for r in res:
        t = 1

    for i in glob.glob(path):
        f = open(i, "r")
        x = f.readlines()
        f.close()
        prot = "0,"
        prot += x[0].split("RES:")[1]
        prot = prot.split(",")
        prot.insert(len(prot)-1, "0")
        prot.pop()
        new_prot = []
        for p in range(len(prot)):
            new_prot.append(prot[p:p+3])
        dssp = (x[1].split("DSSP:")[1]).split(",")
        base.append(tuple(zip(new_prot, dssp)))
    return base

In [0]:
def conversion8to3(value):
    #     helix (G, H and I), strand (E and B) and loop (S, T, and _)
    c = {"H": ["G", "H", "I"], "S": ["E", "B"], "_": ["S", "T", "_", "?"]}
    for t in c.items():
        if value in t[1]:
            return t[0]

    print(value)


def loadfolderproteins(path):
    base = pd.DataFrame()
    res = []
    dssp8 = []
    dssp3 = []
    for i in glob.glob(path):
        f = open(i, "r")
        x = f.readlines()
        f.close()
        prot = x[0].split("RES:")[1]
        prot = prot.split(",")
        prot.pop()
        res.append("".join(prot))
        lb = x[1].split("DSSP:")[1]
        lb = lb.split(",")
        lb.pop()
        lb3 = [conversion8to3(bl) for bl in lb]
        dssp8.append("".join(lb))
        dssp3.append("".join(lb3))

    base["RES"] = res
    base['DSSP8'] = dssp8
    base['DSSP3'] = dssp3
    base.to_csv("NewCB513.csv", index=False)


def load(X, size, window, class="DSSP8"):
    data = X['RES']
    classes = X[class]
    positions = {'A':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'K':9,'L':10,'M':11,'N':12,
                 'P':13,'Q':14,'R':15,'S':16,'T':17,'V':18,'W':19,'Y':20}
#     classes_converter = {'-': 0, 'E':1, 'H':2}
    res = []
    for i in range(len(data)):
        for j in range(len(data[i])):
            res.append(int(positions[data[i][j]]))
    for i in range(size):
        res.insert(0, 0)
        res.append(0)
    data =[]
    for i in range(len(X['prot'])):
        data.append(res[i:window + i])
        data[i].append(classes[i])
#         data[i].append(classes_converter[classes[i]])
    columns = [[]]*(window + 1)
    for i in range(window):
        columns[i] = "Element" + str(i)
    columns[window] = 'Class'
    data = pd.DataFrame(data, columns = columns)
    
    return data


def load_data(window):
    size = int(window/2)
    proteins = pd.read_csv('NewCB513.csv')
    res = load(proteins.iloc[0], size, window)
    for i in range(1,len(proteins)):
        aux = load(proteins.iloc[i], size, window)
        res = res.append(aux,ignore_index = True)
        
    X = res.drop(['Class'], axis=1)
    y = res['Class']
    return X, y

#Functions

In [0]:

def amostra_estrat(tam , df , classe):
    classes = df[classe].unique()
    qtde_por_classe = round(tam / len(classes))
    amostras_por_classe = []
    for c in classes:
        indices_c = df[classe] == c
        obs_c = df[indices_c]
        amostra_c = obs_c.sample(qtde_por_classe)
        amostras_por_classe.append(amostra_c)
    amostra_estratificada = pd.concat(amostras_por_classe)
    return amostra_estratificada
  

arrayfitnessGWO = []
  
def fitnessGWO(chromossome,Xtrain,ytrain,Xval,yval):
    
    w1 = np.split(np.array(chromossome[0:len(chromossome)-6]), 50)
    w2 = np.reshape(chromossome[len(chromossome)-7:len(chromossome)-1], (6, 1))

    model.get_layer('dense1').set_weights((np.array(w1), np.zeros(6)))
    model.get_layer('dense2').set_weights((np.array(w2), np.zeros(1)))
    model.fit(Xtrain, ytrain, epochs=20, batch_size=256, verbose = False)
    retorno = model.evaluate(Xval,yval, verbose = False)

    if len(arrayfitnessGWO) == 0:
        model_json = model.to_json()
        with open("modelGWO.json", "w") as json_file:
            json_file.write(model_json)
        model.save_weights("modelGWO.h5")

    atual = retorno[1]

    if len(arrayfitnessGWO) > 1:
        if atual > arrayfitnessGWO[np.argmax(arrayfitnessGWO)]:
        model_json = model.to_json()
        with open("modelGWO.json", "w") as json_file:
            json_file.write(model_json)
            model.save_weights("modelGWO.h5")

    arrayfitnessGWO.append(atual)

    return atual

def loadmodelGWO():
    json_file = open('modelGWO.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("modelGWO.h5")
    loaded_model.compile( loss='binary_crossentropy', optimizer="adam", 
              metrics=['accuracy'])

    return loaded_model


from SwarmPackagePy.intelligence import sw


class gwo(sw):

    def __init__(self, n, function, lb, ub, dimension, iteration,Xtrain,ytrain,Xval,yval,init = None):


        super(gwo, self).__init__()

        if init is None:
            self.__agents = np.random.uniform(lb, ub, (n, dimension))
        else:
            self.__agents = np.array(init)
            
        self._points(self.__agents)
        alpha, beta, delta = self.__get_abd(n, function,Xtrain,ytrain,Xval,yval)

        Gbest = alpha

        for t in range(iteration):

            a = 2 - 2 * t / iteration

            r1 = np.random.random((n, dimension))
            r2 = np.random.random((n, dimension))
            A1 = 2 * r1 * a - a
            C1 = 2 * r2

            r1 = np.random.random((n, dimension))
            r2 = np.random.random((n, dimension))
            A2 = 2 * r1 * a - a
            C2 = 2 * r2

            r1 = np.random.random((n, dimension))
            r2 = np.random.random((n, dimension))
            A3 = 2 * r1 * a - a
            C3 = 2 * r2

            Dalpha = abs(C1 * alpha - self.__agents)
            Dbeta = abs(C2 * beta - self.__agents)
            Ddelta = abs(C3 * delta - self.__agents)

            X1 = alpha - A1 * Dalpha
            X2 = beta - A2 * Dbeta
            X3 = delta - A3 * Ddelta
            self.__agents = (X1 + X2 + X3) / 3

            self.__agents = np.clip(self.__agents, lb, ub)
            self._points(self.__agents)

            alpha, beta, delta = self.__get_abd(n, function,Xtrain,ytrain,Xval,yval)
            if function(alpha,Xtrain,ytrain,Xval,yval) > function(Gbest,Xtrain,ytrain,Xval,yval):
                Gbest = alpha

        self._set_Gbest(Gbest)
        alpha, beta, delta = self.__get_abd(n, function,Xtrain,ytrain,Xval,yval)
        self.__leaders = list(alpha), list(beta), list(delta)

    def __get_abd(self, n, function,Xtrain,ytrain,Xval,yval):

        result = []
        fitness = [(function(self.__agents[i],Xtrain,ytrain,Xval,yval), i) for i in range(n)]
        fitness.sort(reverse=True)

        for i in range(3):
            result.append(self.__agents[fitness[i][1]])

        return result

    def get_leaders(self):

        return list(self.__leaders)

#Main

In [0]:
ite = 35    # Quantidade de gerações dos algoritmos evolutivos 
iteKfold = 5 # Quantidade de iterações que vai ser executado o cross validation de k = 10
scoresfinalMLP = [] # Array para salvar os score de validação da MLP
cont = 0 # Seed para o shuffle da base de dados
popu = 10 # Tamanho da população
kvalue = 10

testGWO = []

k = KFold(kvalue,True,1) # Instancia da classe Kfold com k = 10

for x in range(iteKfold): # For da quantidade de vezes que ira ser executado o cross validation de k = 10

    for train_index,test_index in k.split(dataset): # For do Kfold com K = 10
        X_train, X_test = np.array(dataset)[train_index], np.array(dataset)[test_index] # Separando o conjunto de Treino separado a cima em treino e validação
        y_train, y_test = np.array(labels)[train_index], np.array(labels)[test_index] # Separando o conjunto de Treino separado a cima em treino e validação

        X_trainK, X_valK, y_trainK, y_valK = train_test_split(X_train, y_train, test_size=0.2)


        ############ CREATE POPU ######

        inicialpopulation = np.array(creatpopuMSRA(popu,306)) # Criando a população inicial
        inicialstd = np.array(creatpopuSTD(popu,306)) # Criando a população de desvio padrões iniciais
        sizepop = inicialpopulation.shape[0] # Tamanho da população
        dimpop = inicialpopulation.shape[1] # Dimensão da população

        ############ CREATE POPU #########

        ######## MLP ############

        timeinicialMLP = time.time() # Instanciando o tempo de inicio da execução do MLP
        modelMLP = Sequential() # Instanciando o modelo
        modelMLP.add(Dense(6, input_dim = 50, activation="relu")) # Instanciando o modelo
        modelMLP.add(Dense(1, activation="sigmoid")) # Instanciando o modelo
        modelMLP.compile( loss='binary_crossentropy', optimizer="adam",metrics=['accuracy']) # Instanciando o modelo

        modelMLP.fit(X_trainK, y_trainK, epochs=20, batch_size=256, verbose = False) # Treinando o modelo
        scoresfinalMLP.append(modelMLP.evaluate(X_valK,y_valK,verbose = False)[1]) # Avaliando o modelo com o conjunto de validação e guardando no array de validações

        timefinalMLP = time.time() # pegando o tempo de termino
        timeMLP.append(timefinalMLP-timeinicialMLP) # Salvando o tempo de execução
    
    
    ################ GWO ############################
    
    timeinicialgwo = time.time()
    grey = gwo(sizepop,fitnessGWO,-1,1,dimpop,ite,X_trainK,y_trainK,X_valK,y_valK,init=inicialpopulation.copy(),)# Rodando o GWO com os conjuntos de validação e treinamento e com o número
                                                                                                          # de iterações declaradas no inicio
    timefinalgwo = time.time()
    timeGWO.append(timefinalgwo - timeinicialgwo)
        
    
    modelGWO = loadmodelGWO() # Carregando o melhor modelo gerado pelo GWO para teste
      
    testGWO.append(modelGWO.evaluate(X_test,y_test, verbose = False)[1]) # Avaliando o melhor modelo do GWO com o conjunto de teste

    arrayfitnessGWO = []

    dataset, labels = shuffle(dataset, labels, random_state=cont) # Depois que termina as 10 iterações de um Kfold dou shuffle
    cont += 1 # Valor da seed do shufle sendo somado +1
    